# 모든 모델의 데이터 임베딩 할 때

In [ ]:
# !pip install sentence_transformers

import torch
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import random

# SBERT 모델 불러오기
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
# n1_model_data
n1_model_data = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot/data/csv/n1_model_data.csv',header=0,encoding='cp949')
n1_model_data

In [ ]:
# n2_model_data
n2_model_data = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot/data/csv/n2_model_data.csv',header=0,encoding='cp949')
n2_model_data

In [ ]:
# n3_model_data
n3_model_data = pd.read_csv('/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot/data/csv/n3_model_data.csv',header=0,encoding='cp949')
n3_model_data

In [ ]:
all_n_model_data = pd.concat([n1_model_data,n2_model_data,n3_model_data])
all_n_model_data

In [ ]:
!pip install konlpy
from konlpy.tag import Komoran
tokenizer = Komoran()

In [ ]:
# data의 모든 사용자 input문장을 형태소/품사로 분해하여 리스트에 저장

input_tokenized = [[token+"/"+POS for token, POS in tokenizer.pos(input)] for input in  n_model_data['input']]

In [ ]:
len(input_tokenized)

In [ ]:
input_tokenized[5]

In [ ]:
exclusion_tags = [
    'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            'JX', 'JC',
            'SF', 'SP', 'SS', 'SE', 'SO',
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            'XSN', 'XSV', 'XSA'
]

f = lambda x: x in exclusion_tags

core_input_tokenized = []

# i는 문장의 갯수
for i in range(len(input_tokenized)):
        temp = []
        # data_tokenized[0]=> 한 문장
        # j는 한 문장을 구성하는 '단어/품사' 갯수
        for j in range(len(input_tokenized[i])):
            # f에 품사를 넣음
            # exclusion_tags 리스트에 포함되지 않는 품사면 append!
            # 필요없는 품사가 아니면 append
            # (조사 같은 것들은 제외!)
            if f(input_tokenized[i][j].split('/')[1]) is False:
                temp.append(input_tokenized[i][j].split('/')[0])
        core_input_tokenized.append(temp)

In [ ]:
input_tokenized[1] , core_input_tokenized[1]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 불용어들을 제외한 '의미있는' 단어들의 길이들을 모아놓은 num_tokens
num_tokens = [len(tokens) for tokens in core_input_tokenized]
num_tokens = np.array(num_tokens)

# 평균값, 최댓값, 표준편차
print(f"토큰 길이 평균: {np.mean(num_tokens)}")
print(f"토큰 길이 최대: {np.max(num_tokens)}")
print(f"토큰 길이 표준편차: {np.std(num_tokens)}")

plt.title('all text length')
plt.hist(num_tokens, bins=100)
# bins는 가로축 구간의 개수
# (막대 그래프의 갯수)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
import pickle

class Preprocess:
    def __init__(self, word2index_dic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/love_chatbot_sbert_v2(custom_data)/data/chatbot_dict/chatbot_dict.bin' ,userdic=None): # userdic 인자에는 사용자 정의 사전 파일 경로 입력가능
        # 단어 인덱스 사전 불러오기 추가
        if(word2index_dic != ''):
            f = open(word2index_dic, "rb")
            self.word_index = pickle.load(f)
            f.close()
            print("단어 사전 로드 완료..")
        else:
            self.word_index = None
            print("단어 사전 로드 실패..")

        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            # 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사
            'JX', 'JC',
            # 보조사, 접속조사
            'SF', 'SP', 'SS', 'SE', 'SO',
            # 마침표,물음표,느낌표(SF), 쉼표,가운뎃점,콜론,빗금(SP), 따옴표,괄호표,줄표(SS), 줄임표(SE), 붙임표(물결,숨김,빠짐)(SO)
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            # 선어말어미, 종결어미, 연결어미, 명사형전성어미, 관형형전성어미
            'XSN', 'XSV', 'XSA'
            # 명사파생접미사, 동사파생접미사, 형용사파생접미사
        ]


    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # 불용어 제거 후 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list

    # 키워드를 단어 인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                # 해당 단어가 사전에 없는 경우 OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i

p = Preprocess(word2index_dic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/love_chatbot_sbert_v2(custom_data)/data/chatbot_dict/chatbot_dict.bin',
               userdic=None)

In [ ]:
inputs = n_model_data['input'].tolist()

In [ ]:
inputs[2]

In [ ]:
!pip install tqdm
from tqdm import tqdm

sequences = []
check_keywords = True
# text는 모든 문장들의 list
for sentence in tqdm(inputs):

    # 문장을 [(단어1,품사1),(단어2,품사2)...] 로 변환
    pos = p.pos(sentence)

    # get_keywords(pos, without_tag=True) => 불용어 처리 후 품사(태그)없이 단어들만의 list
    # keywords : 불용어 처리된 [(단어1,품사1),(단어2,품사2)...], list형
    keywords = p.get_keywords(pos, without_tag=True)

    # 첫번째 keywords 와 sequence[0] 어떻게 대응되는지 체크해보고 싶음
    if check_keywords is True:
      print(keywords)
      check_keywords = False
    # 태그없이 '단어'만 있는 keywords에서 [[단어1,단어2],[단어1,단어2,단어3]...]들을 인덱싱해줌
    # 우리가 만든 단어사전에 없으면(OOV token이므로 인덱스 1로 고정)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [ ]:
MAX_SEQ_LEN = 10
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

In [ ]:
len(padded_seqs[3]) ,len(padded_seqs[1]),len(padded_seqs[5])

In [ ]:
all_n_model_data

In [ ]:
!pip install -U sentence-transformers
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
all_n_model_data['embedding_vector'] = all_n_model_data['input'].progress_map(lambda x : model.encode(x))
all_n_model_data.head()

In [ ]:
all_n_model_data.to_excel("/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot/data/csv/all_n_model_data_embedding.xlsx", index=False)
all_n_model_data

In [ ]:
all_n_model_data_embedding_data = torch.tensor(n_model_data['embedding_vector'].tolist())
torch.save(all_n_model_data_embedding_data, '/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot/data/embedding_data/all_n_model_data_embedding_data.pt')

# 각 모델의 데이터 임베딩 할 때

In [ ]:
!pip install -U sentence-transformers
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import torch
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [2]:
%cd /content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot

# n1_model_data
n1_model_data = pd.read_csv('./data/csv/n1_model_data.csv',header=0,encoding='cp949')
n1_model_data

/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_최종프로젝트/tree_chatbot


,input,output
0,저 예약좀 하려고요,네 예약 도와드리겠습니다
1,저희 예약하려고요,네 예약 도와드리겠습니다
2,예약 가능 할까요?,네 예약 도와드리겠습니다


In [3]:
# n2_model_data
n2_model_data = pd.read_csv('./data/csv/n2_model_data.csv',header=0,encoding='cp949')
n2_model_data

,input,output
0,이번주 삼월일일 열두시에 다섯명이요,네 잠시만요 확인해드리겠습니다.
1,이번주 삼월일일 열두시로 예약해주세요,네 인원 수는 몇 명이신가요?
2,저희 다섯명 예약해주세요,네 날짜와 시간은 어떻게 되나요?


In [4]:
# n3_model_data
n3_model_data = pd.read_csv('./data/csv/n3_model_data.csv',header=0,encoding='cp949')
n3_model_data

,input,output
0,그럼 그 다음날 예약 가능할까요?,네 잠시만요 확인해드리겠습니다.
1,그럼 다음주 수요일에 예약 되나요?,확인해드리겠습니다. 잠시만요


In [6]:
list_n_model_data = [n1_model_data,n2_model_data,n3_model_data]

In [ ]:
!pip install konlpy
from konlpy.tag import Komoran
tokenizer = Komoran()

In [7]:
n1_input_tokenized = []
n2_input_tokenized = []
n3_input_tokenized = []

# 아직 비어있음
list_input_tokenized = [n1_input_tokenized, n2_input_tokenized, n3_input_tokenized ]

In [10]:
# data의 모든 사용자 input문장을 형태소/품사로 분해하여 리스트에 저장

for i in range(len(list_input_tokenized)):
  list_input_tokenized[i] = [[token+"/"+POS for token, POS in tokenizer.pos(input)] for input in  list_n_model_data[i]['input']]

In [16]:
len(list_input_tokenized[0]) , list_input_tokenized[0][:]

(3,
 [['저/MM', '예약/NNG', '좀/MAG', '하/VV', '려고/EC', '요/JX'],
  ['저희/NP', '예약/NNG', '하/XSV', '려고/EC', '요/JX'],
  ['예약/NNG', '가능/XR', '하/VX', 'ㄹ까요/EF', '?/SF']])

-- 최대 토큰 길이 생략

In [17]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
import pickle

class Preprocess:
    def __init__(self, word2index_dic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/love_chatbot_sbert_v2(custom_data)/data/chatbot_dict/chatbot_dict.bin' ,userdic=None): # userdic 인자에는 사용자 정의 사전 파일 경로 입력가능
        # 단어 인덱스 사전 불러오기 추가
        if(word2index_dic != ''):
            f = open(word2index_dic, "rb")
            self.word_index = pickle.load(f)
            f.close()
            print("단어 사전 로드 완료..")
        else:
            self.word_index = None
            print("단어 사전 로드 실패..")

        # 형태소 분석기 초기화
        self.komoran = Komoran(userdic=userdic)

        # 제외할 품사
        # 참조 : https://docs.komoran.kr/firststep/postypes.html
        # 관계언 제거, 기호 제거
        # 어미 제거
        # 접미사 제거
        self.exclusion_tags = [
            'JKS', 'JKC', 'JKG', 'JKO', 'JKB', 'JKV', 'JKQ',
            # 주격조사, 보격조사, 관형격조사, 목적격조사, 부사격조사, 호격조사, 인용격조사
            'JX', 'JC',
            # 보조사, 접속조사
            'SF', 'SP', 'SS', 'SE', 'SO',
            # 마침표,물음표,느낌표(SF), 쉼표,가운뎃점,콜론,빗금(SP), 따옴표,괄호표,줄표(SS), 줄임표(SE), 붙임표(물결,숨김,빠짐)(SO)
            'EP', 'EF', 'EC', 'ETN', 'ETM',
            # 선어말어미, 종결어미, 연결어미, 명사형전성어미, 관형형전성어미
            'XSN', 'XSV', 'XSA'
            # 명사파생접미사, 동사파생접미사, 형용사파생접미사
        ]


    # 형태소 분석기 POS 태거
    def pos(self, sentence):
        return self.komoran.pos(sentence)

    # 불용어 제거 후 필요한 품사 정보만 가져오기
    def get_keywords(self, pos, without_tag=False):
        f = lambda x: x in self.exclusion_tags
        word_list = []
        for p in pos:
            if f(p[1]) is False:
                word_list.append(p if without_tag is False else p[0])
        return word_list

    # 키워드를 단어 인덱스 시퀀스로 변환
    def get_wordidx_sequence(self, keywords):
        if self.word_index is None:
            return []
        w2i = []
        for word in keywords:
            try:
                w2i.append(self.word_index[word])
            except KeyError:
                # 해당 단어가 사전에 없는 경우 OOV 처리
                w2i.append(self.word_index['OOV'])
        return w2i

p = Preprocess(word2index_dic='/content/drive/MyDrive/새싹_인공지능SW교육/프로젝트/새싹_자연어처리프로젝트/love_chatbot_sbert_v2(custom_data)/data/chatbot_dict/chatbot_dict.bin',
               userdic=None)

단어 사전 로드 완료..


In [18]:
list_n_model_data 

[        input         output
 0  저 예약좀 하려고요  네 예약 도와드리겠습니다
 1   저희 예약하려고요  네 예약 도와드리겠습니다
 2  예약 가능 할까요?  네 예약 도와드리겠습니다,                   input              output
 0   이번주 삼월일일 열두시에 다섯명이요   네 잠시만요 확인해드리겠습니다.
 1  이번주 삼월일일 열두시로 예약해주세요    네 인원 수는 몇 명이신가요?
 2         저희 다섯명 예약해주세요  네 날짜와 시간은 어떻게 되나요?,                  input             output
 0   그럼 그 다음날 예약 가능할까요?  네 잠시만요 확인해드리겠습니다.
 1  그럼 다음주 수요일에 예약 되나요?    확인해드리겠습니다. 잠시만요]

In [ ]:
!pip install tqdm
!pip install -U sentence-transformers
from tqdm import tqdm
import torch
from sentence_transformers import SentenceTransformer


for i in range(len(list_n_model_data)):

  inputs = list_n_model_data[i]['input'].tolist()

  sequences = []
  check_keywords = True
  # text는 모든 문장들의 list
  for sentence in tqdm(inputs):

      # 문장을 [(단어1,품사1),(단어2,품사2)...] 로 변환
      pos = p.pos(sentence)

      # get_keywords(pos, without_tag=True) => 불용어 처리 후 품사(태그)없이 단어들만의 list
      # keywords : 불용어 처리된 [(단어1,품사1),(단어2,품사2)...], list형
      keywords = p.get_keywords(pos, without_tag=True)

      # 첫번째 keywords 와 sequence[0] 어떻게 대응되는지 체크해보고 싶음
      if check_keywords is True:
        print(keywords)
        check_keywords = False
      # 태그없이 '단어'만 있는 keywords에서 [[단어1,단어2],[단어1,단어2,단어3]...]들을 인덱싱해줌
      # 우리가 만든 단어사전에 없으면(OOV token이므로 인덱스 1로 고정)
      seq = p.get_wordidx_sequence(keywords)
      sequences.append(seq)

      MAX_SEQ_LEN = 10
      padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')


      tqdm.pandas()
      model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

      list_n_model_data[i]['embedding_vector'] = list_n_model_data[i]['input'].progress_map(lambda x : model.encode(x))
      list_n_model_data[i].to_excel("./data/csv/n" + str(i+1) + "_model_data_embedding.xlsx", index=False)
      n_model_data_embedding_data = torch.tensor(list_n_model_data[i]['embedding_vector'].tolist())
      torch.save(n_model_data_embedding_data, "./data/embedding_data/n" + str(i+1) + "_model_data_embedding_data.pt")
